In [29]:
import numpy as np
from random import shuffle
from treys import Card, Deck, Evaluator
    
class Game:
    def __init__(self):
        self.player_count = 2
        self.deck = Deck()
        self.player = Role('Player')
        self.dealer = Role('Dealer')
        self.dealer_chips = []
        self.player_chips = []
        self.next_state = 'start'
    # state: start, round1, round2, round3, round4, accounting, abstain, end
    def go_next_state(self):
        if self.next_state == 'start':
            self.start()
        elif self.next_state == 'round1':
            self.round1()
        elif self.next_state == 'round2':
            self.round2()
        elif self.next_state == 'round3':
            self.round3()
        elif self.next_state == 'round4':
            self.round4()
        elif self.next_state == 'accounting':
            self.accounting()
        elif self.next_state == 'abstain':
            self.abstain()
        elif self.next_state == 'end':
            self.end()
    def start(self):
        self.board = Board()
        self.dispatch_card(False)
        self.dealer_chips = []
        self.player_chips = []
        if self.player.chips_count > 0 and self.dealer.chips_count > 0:
            self.next_state = 'round1'
        else:
            self.next_state = 'end'
        print('== start game ==')
        self.go_next_state()
        
    # return is_continue or abstain
    def wait_bet(self):
        def add_dealer_chips(count):
            # no strategy for dealer now
            available_chips = self.dealer.chips_count - sum(self.dealer_chips)
            if count > available_chips:
                self.dealer_chips.append(available_chips) 
            else:
                self.dealer_chips.append(count)
        while(1):
            chips_on_table = sum(self.player_chips)
            available = self.player.chips_count - chips_on_table
            if available == 0:
                print('already show hand')
                return True
            input_char = input('How much?  a: abstain , e: exit game ')
            if input_char == 'a':
                self.next_state = 'abstain'
                return False
            elif input_char == 'e':
                self.next_state = 'end'
                return False
            else:
                try:
                    bet_this_round = int(input_char)
                    if (bet_this_round > available):
                        print('not enough chips')
                    elif (bet_this_round == available):
                        self.player_chips.append(bet_this_round)
                        add_dealer_chips(bet_this_round)
                        print('show hand!!')
                        return True
                    else:
                        self.player_chips.append(bet_this_round)
                        add_dealer_chips(bet_this_round)
                        return True
                except ValueError:
                    print('not a valid int number')
                    
    def round1(self):
        self.dispatch_card(True)
        self.print_board()
        if self.wait_bet():
            self.next_state = 'round2'
        self.go_next_state()
    def round2(self):
        self.dispatch_card(True)
        self.print_board()
        if self.wait_bet():
            self.next_state = 'round3'
        self.go_next_state()
    def round3(self):
        self.dispatch_card(True)
        self.print_board()
        if self.wait_bet():
            self.next_state = 'round4'
        self.go_next_state()
    def round4(self):
        self.dispatch_card(True)
        self.print_board()
        if self.wait_bet():
            self.next_state = 'accounting'
        self.go_next_state()
    def abstain(self):
        self.print_board(True)
        print('Losserrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr!!!!!!!!!!')
        player_chips_on_board = sum(self.player_chips)
        self.player.chips_count -= player_chips_on_board / 2
        self.next_state = 'start'
        self.go_next_state()
    def accounting(self):
        self.print_board(True)
        player_cards = self.board.player_private_card + self.board.player_public_card
        dealer_cards = self.board.dealer_private_card + self.board.dealer_public_card
        evaluator = Evaluator()
        player_rank = evaluator.evaluate(player_cards, [])
        dealer_rank = evaluator.evaluate(dealer_cards, [])
        if player_rank < dealer_rank:
            print('You WIN!')
            dealer_chips_on_board = sum(self.dealer_chips)
            self.player.chips_count += dealer_chips_on_board
            self.dealer.chips_count -= dealer_chips_on_board
        else:
            print('You LOSER!')
            player_chips_on_board = sum(self.player_chips)
            self.player.chips_count -= player_chips_on_board
            self.dealer.chips_count += player_chips_on_board
        self.next_state = 'start'
        self.go_next_state()
    def end(self):
        print('end game')
    def dispatch_card(self, is_public):
        if is_public:
            self.board.dealer_public_card.append(self.deck.draw())
            self.board.player_public_card.append(self.deck.draw())
        else:
            self.board.dealer_private_card.append(self.deck.draw())
            self.board.player_private_card.append(self.deck.draw())
    def print_board(self, is_print_all=False):
        output = 'dealer: {0}  player: {1} \n'.format(self.dealer.chips_count, self.player.chips_count)
        dealer = 'd: '
        if is_print_all:
            dealer += str(Card.int_to_pretty_str(self.board.dealer_private_card[0]))
        else:
            dealer += '[?？]'
        dealer += ','.join([str(Card.int_to_pretty_str(card)) for card in self.board.dealer_public_card]) + '\n'
        player = 'p: ' + str(Card.int_to_pretty_str(self.board.player_private_card[0])) + ','.join([str(Card.int_to_pretty_str(card)) for card in self.board.player_public_card]) + '\n'
        dealer_bet = 'dealer: {0} \n'.format(sum(self.dealer_chips))
        player_bet = 'player: {0} \n'.format(sum(self.player_chips))
        print('========== \n' + output + dealer + player + dealer_bet + player_bet + '=========')

class Board():
    def __init__(self):
        self.dealer_private_card = []
        self.dealer_public_card = []
        self.player_private_card = []
        self.player_public_card = []
        
class Role:
    def __init__(self, name):
        self.name = name
        self.chips_count = 10000


In [30]:
g = Game()
g.start()

== start game ==
dealer: 10000  player: 10000 
d: [?？][A♠]
p: [Q♦][J♣]
dealer: 0 
player: 0 
How much?  a: abstain , e: exit game 100
dealer: 10000  player: 10000 
d: [?？][A♠],[6♠]
p: [Q♦][J♣],[8♦]
dealer: 100 
player: 100 
How much?  a: abstain , e: exit game 100
dealer: 10000  player: 10000 
d: [?？][A♠],[6♠],[K♠]
p: [Q♦][J♣],[8♦],[5♦]
dealer: 200 
player: 200 
How much?  a: abstain , e: exit game 100
dealer: 10000  player: 10000 
d: [?？][A♠],[6♠],[K♠],[4♣]
p: [Q♦][J♣],[8♦],[5♦],[3♦]
dealer: 300 
player: 300 
How much?  a: abstain , e: exit game 1
dealer: 10000  player: 10000 
d: [9♦][A♠],[6♠],[K♠],[4♣]
p: [Q♦][J♣],[8♦],[5♦],[3♦]
dealer: 301 
player: 301 
You LOSER!
== start game ==
dealer: 10301  player: 9699 
d: [?？][J♦]
p: [T♣][J♠]
dealer: 0 
player: 0 
How much?  a: abstain , e: exit game e
end game
